In [55]:
import pandas as pd


from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,Attention
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [56]:
dataset=pd.read_csv("IMDB Dataset.csv")

In [57]:
dataset.replace ({"sentiment":{"positive":1,"negative":0}}, inplace=True)
dataset.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_11780\3517365726.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.replace ({"sentiment":{"positive":1,"negative":0}}, inplace=True)


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [58]:
dataset.duplicated().sum()

np.int64(418)

In [59]:
dataset.drop_duplicates(inplace=True)
dataset.duplicated().sum()

np.int64(0)

In [60]:
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
print(train_data.shape, test_data.shape)

(39665, 2) (9917, 2)


In [61]:
tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(train_data["review"])

x_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [62]:

y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

In [63]:
embedding_dim=32
vocab_size=5000

model = Sequential([
    Embedding(vocab_size, embedding_dim, name="embedding"),
    Bidirectional(LSTM(64)),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

In [64]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
history = model.fit(x_train, y_train, epochs=3, batch_size=32, validation_split=0.2)

Epoch 1/3
992/992 ━━━━━━━━━━━━━━━━━━━━ 51s 48ms/step - accuracy: 0.7482 - loss: 0.4938 - val_accuracy: 0.8710 - val_loss: 0.3297
Epoch 2/3
992/992 ━━━━━━━━━━━━━━━━━━━━ 48s 48ms/step - accuracy: 0.8880 - loss: 0.2809 - val_accuracy: 0.8456 - val_loss: 0.3719
Epoch 3/3
534/992 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - accuracy: 0.9140 - loss: 0.2251

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

310/310 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8808 - loss: 0.2852
Test Loss: 0.2943582534790039
Test Accuracy: 0.8803065419197083


In [ ]:
def predict_sentiment(review):

  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)


  prediction = model.predict(padded_sequence)
  print(prediction)


  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [ ]:
while True:
  new_review = input("Enter a movie review (or type 'exit' to quit): ")
  print(new_review)
  if new_review.lower() == "exit":
    break
  sentiment = predict_sentiment(new_review)
  print("Sentiment of the review is:", sentiment)

Good


ValueError: Attention layer must be called on a list of inputs, namely [query, value] or [query, value, key]. Received: inputs=(1, 128).